In [1]:
!pip install datasets
!pip install transformers
!sudo apt-get install swig
!sudo pip install jamspell
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 26.2 MB/s 
     |████████████████████████████████| 182 kB 68.3 MB/s 
     |████████████████████████████████| 212 kB 40.0 MB/s 
     |████████████████████████████████| 132 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 67.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.3 MB/s 
     |████████████████████████████████| 7.6 MB 39.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
T

In [2]:
import string
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import jamspell
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import torch
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, set_seed

from sklearn.metrics import classification_report
from scipy.special import softmax
from tqdm import tqdm

import datasets
from datasets import load_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
# Descargamos los datasets a utilizar de hugginface.
dataset = datasets.load_dataset('tweet_eval', 'sentiment')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# DataFrame format.
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

In [5]:
# Modelo a utilizar
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [6]:
# Iniciamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True, max_length=512)
#config = AutoConfig.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [7]:
# Inicializamos nuestro modelo.
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
import operator
#Funcion para vectorizar, entrenar el modelo y sacar el output del texto.

def polarity_scores(example):
  # vectorizamos el texto
  encoded_text = tokenizer(example, return_tensors='pt')
  # aplicamos el modelo
  output = model(**encoded_text)
  # Cogemos los scores de las labels
  scores = output[0][0].detach().numpy()
  # Normalizamos los scores
  scores = softmax(scores)
  # creamos un diccionario con las label y sus scores
  scores_dict = {
      '0' : scores[0],
      '1' : scores[1],
      '2' : scores[2]
  }
  # Obtenemos la label con mayor score
  final_label = max(scores_dict.items(), key=operator.itemgetter(1))[0]
  
  return final_label, scores_dict

##1. Hard Cleaning


In [9]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')
lemmatizer= WordNetLemmatizer()
punct = ''.join([i for i in string.punctuation if i not in [',','?','!','.', '#']])

In [10]:
def hard_clean(x):
  # normalizacion del texto es minusculas.
  x = x.lower()
  x = x.encode('ascii', 'ignore').decode()
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos @usuarios.
  x = re.sub(r'@[\w]+', '', x)
  # Removemos Hashtag
  x = re.sub(r'#[\w]+', '', x)
  # Removemos los signos de puntuacion.
  x = re.sub('[^\w\s]', '', x)
  # Spell check
  x = corrector.FixFragment(x)
  # Removemos palabras con menos de dos caracteres.
  x = ' '.join([w for w in x.split() if len(w)>2])
  # Lematizacion y stop words removal
  x = ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stopwords.words('english')])
  return x

In [11]:
test_df['text_c1'] = test_df['text'].apply(hard_clean)
test_df['text_c1'][0]

'naked pic anything even like'

In [12]:
resultados_c1 = [polarity_scores(line) for line in test_df['text_c1']]

In [13]:
predictions_c1 = [int(i[0]) for i in resultados_c1]

In [14]:
report_c1 = classification_report(test_df['label'], predictions_c1, output_dict=True)
report_c1

{'0': {'precision': 0.6838559212231148,
  'recall': 0.6644008056394763,
  'f1-score': 0.6739879964244668,
  'support': 3972},
 '1': {'precision': 0.6520790927595231,
  'recall': 0.7554320363820111,
  'f1-score': 0.6999609832227857,
  'support': 5937},
 '2': {'precision': 0.7310924369747899,
  'recall': 0.4762105263157895,
  'f1-score': 0.5767465578786333,
  'support': 2375},
 'accuracy': 0.6720123738196028,
 'macro avg': {'precision': 0.6890091503191426,
  'recall': 0.6320144561124256,
  'f1-score': 0.6502318458419619,
  'support': 12284},
 'weighted avg': {'precision': 0.6776305625713633,
  'recall': 0.6720123738196028,
  'f1-score': 0.6677402925881972,
  'support': 12284}}

##2. Mid clean

In [15]:
def mid_clean(x):
  # normalizacion del texto es minusculas.
  x = x.lower()
  x = x.encode('ascii', 'ignore').decode()
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos las palabras que empiezan por @.
  x = re.sub(r'@[\w]+', '', x)
  # Removemos los signos de puntuacion.
  x = re.sub('[^\w\s]', '', x)
  # Spell check
  x = corrector.FixFragment(x)
  # Removemos palabras con menos de dos caracteres.
  x = ' '.join([w for w in x.split() if len(w)>2])
  return x

In [16]:
test_df['text_c2'] = test_df['text'].apply(mid_clean)
test_df['text_c2'][0]

'what these naked pics have with anything there not even like that'

In [17]:
resultados_c2 = [polarity_scores(line) for line in test_df['text_c2']]

In [18]:
predictions_c2 = [int(i[0]) for i in resultados_c2]

In [19]:
report_c2 = classification_report(test_df['label'], predictions_c2, output_dict=True)
report_c2

{'0': {'precision': 0.6676796179726503,
  'recall': 0.7744209466263847,
  'f1-score': 0.7170998950926681,
  'support': 3972},
 '1': {'precision': 0.6950732356857523,
  'recall': 0.7033855482566953,
  'f1-score': 0.6992046881540394,
  'support': 5937},
 '2': {'precision': 0.7639304973037747,
  'recall': 0.5368421052631579,
  'f1-score': 0.6305637982195846,
  'support': 2375},
 'accuracy': 0.6941549983718659,
 'macro avg': {'precision': 0.7088944503207258,
  'recall': 0.6715495333820792,
  'f1-score': 0.682289460488764,
  'support': 12284},
 'weighted avg': {'precision': 0.699528506508478,
  'recall': 0.6941549983718659,
  'f1-score': 0.6917199639897528,
  'support': 12284}}

##3. Light clean

In [20]:
def light_clean(x):
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos las palabras que empiezan por @.
  x = re.sub(r'@[\w]+', '', x)
  # Signos de puntuacion menos. 
  x = x.translate(str.maketrans('', '', punct))
  x = x.replace('#', '')
  # Spell check
  x = corrector.FixFragment(x)
  return x

In [21]:
test_df['text_c3'] = test_df['text'].apply(light_clean)
test_df['text_c3'][0]

'  what do these 12 naked pics have to do with anything? There not even like that.'

In [22]:
resultados_c3 = [polarity_scores(line) for line in test_df['text_c3']]

In [23]:
predictions_c3 = [int(i[0]) for i in resultados_c3]

In [24]:
report_c3 = classification_report(test_df['label'], predictions_c3, output_dict=True)
report_c3

{'0': {'precision': 0.6792735042735043,
  'recall': 0.8003524672708963,
  'f1-score': 0.7348589921405456,
  'support': 3972},
 '1': {'precision': 0.7441193344807803,
  'recall': 0.6553815058110156,
  'f1-score': 0.6969371305749597,
  'support': 5937},
 '2': {'precision': 0.7115789473684211,
  'recall': 0.7115789473684211,
  'f1-score': 0.7115789473684211,
  'support': 2375},
 'accuracy': 0.7131227613155324,
 'macro avg': {'precision': 0.711657262040902,
  'recall': 0.7224376401501109,
  'f1-score': 0.714458356694642,
  'support': 12284},
 'weighted avg': {'precision': 0.7168602122913345,
  'recall': 0.7131227613155324,
  'f1-score': 0.7120299300721087,
  'support': 12284}}